In [1]:
import os
import pathlib
import unittest

import healpy as hp
import numpy as np

from rubin_scheduler.data import get_data_dir
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.example import example_scheduler, run_sched
from rubin_scheduler.scheduler.model_observatory import KinemModel, ModelObservatory
from rubin_scheduler.scheduler.utils import (
    ObservationArray,
    ScheduledObservationArray,
    SchemaConverter,
    ecliptic_area,
    make_rolling_footprints,
    restore_scheduler,
    run_info_table,
    season_calc,
)
from rubin_scheduler.utils import SURVEY_START_MJD



In [2]:
"""Test we can restore a scheduler properly"""
# MJD set so it's in test data range
mjd_start = SURVEY_START_MJD
n_visit_limit = 3000

scheduler = example_scheduler(survey_start_mjd=mjd_start)

mo = ModelObservatory(mjd_start=mjd_start, downtimes="ideal", cloud_data="ideal")
# Never load too many nights of sky
mo.sky_model.load_length = 10.0
mo, scheduler, observations = sim_runner(
    mo,
    scheduler,
    sim_duration=30.0,
    verbose=False,
    filename=None,
    n_visit_limit=n_visit_limit,
)

/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 463 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 463 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 162 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 162 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/git_repos/rubin_scheduler/rubin_scheduler/utils/healpy_utils.py:339: 

Skipped 0 observations
Flushed 0 observations from queue for being stale
Completed 3000 observations
ran in 0 min = 0.0 hours


In [3]:
# Won't be exact if we restart in the middle of a blob sequence
# since the queue isn't reconstructed.
# Also, any scripted observations that get generated
# during the night (e.g., long gaps observations) will get lost,
# so need to restart on a new night to ensure identical results.

nd = np.zeros(observations.size)
nd[1:] = np.diff(observations["night"])

break_indx = np.min(np.where((observations["ID"] >= n_visit_limit / 2.0) & (nd != 0))[0])
new_n_limit = n_visit_limit - break_indx

new_mo = ModelObservatory(mjd_start=mjd_start, downtimes="ideal", cloud_data="ideal")
# Never load too much sky
new_mo.sky_model.load_length = 10.0
new_sched = example_scheduler(survey_start_mjd=mjd_start)

# Restore some of the observations
new_sched, new_mo = restore_scheduler(break_indx - 1, new_sched, new_mo, observations, fast=False)



/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 463 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 463 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 162 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 162 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


In [4]:
new_sched.target_id_counter

np.int64(2212)

In [5]:
observations[break_indx]["ID"]

np.int64(1641)

In [6]:
# Simulate ahead and confirm that it behaves the same
# as running straight through
new_mo, new_sched, new_obs = sim_runner(
    new_mo,
    new_sched,
    sim_duration=20.0,
    verbose=False,
    filename=None,
    n_visit_limit=new_n_limit,
)

/Users/yoachim/git_repos/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:359: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/Users/yoachim/git_repos/rubin_scheduler/rubin_scheduler/utils/healpy_utils.py:339: RuntimeWarning: All-NaN slice encountered
  indx = np.where(in_map[current_neighbors] == np.nanmax(in_map[current_neighbors]))[0]
/Users/yoachim/git_repos/rubin_scheduler/rubin_scheduler/utils/healpy_utils.py:342: UserWarning: Can not connect to any more pixels.
  warnings.warn("Can not connect to any more pixels.")


Skipped 0 observations
Flushed 0 observations from queue for being stale
Completed 1359 observations
ran in 0 min = 0.0 hours


In [7]:
# Manually fix the target_id if needed
new_obs["target_id"] = observations[break_indx:]["target_id"]

# Check that observations taken after restart match those from
# before Jenkins can be bad at comparing things, so if it thinks
# they aren't the same, check column-by-column to double check
if not np.all(new_obs == observations[break_indx:]):
    names = new_obs.dtype.names
    for name in names:
        # If it's a string
        if new_obs[name].dtype == "<U40":
            assert np.all(new_obs[name] == observations[break_indx:][name])
        # Otherwise should be number-like
        else:
            assert np.allclose(new_obs[name], observations[break_indx:][name])

# Didn't need to go by column, the observations after restart
# match the ones that were taken all at once.
else:
    assert np.all(new_obs == observations[break_indx:])





In [8]:
new_obs["target_id"] #- observations[break_indx:]["RA"]

ObservationArray([2212, 2213, 2214, ..., 4194, 4195, 4196], shape=(1359,))

In [9]:
new_obs["ID"]

ObservationArray([1641, 1642, 1643, ..., 2997, 2998, 2999], shape=(1359,))

In [10]:
observations[break_indx:]["target_id"]

ObservationArray([2212, 2213, 2214, ..., 4194, 4195, 4196], shape=(1359,))

In [11]:
# And again, but this time using the fast array restore
new_mo = ModelObservatory(mjd_start=mjd_start, downtimes="ideal", cloud_data="ideal")
new_mo.sky_model.load_length = 10.0
new_sched = example_scheduler(survey_start_mjd=mjd_start)
new_sched, new_mo = restore_scheduler(break_indx - 1, new_sched, new_mo, observations, fast=True)
# Simulate ahead and confirm that it behaves the same as
# running straight through
new_mo, new_sched, new_obs_fast = sim_runner(
    new_mo,
    new_sched,
    sim_duration=20.0,
    verbose=False,
    filename=None,
    n_visit_limit=new_n_limit,
)

/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 463 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 463 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 162 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/anaconda3/envs/rubin12_b/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 162 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/yoachim/git_repos/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model

Skipped 0 observations
Flushed 0 observations from queue for being stale
Completed 1359 observations
ran in 0 min = 0.0 hours


In [12]:
# Manually fix the target_id if needed
new_obs["target_id"] = observations[break_indx:]["target_id"]
# Check that observations taken after restart match those
# from before Jenkins can be bad at comparing things, so if
# it thinks they aren't the same, check column-by-column to
# double check
if not np.all(new_obs_fast == observations[break_indx:]):
    names = new_obs_fast.dtype.names
    for name in names:
        # If it's a string
        if new_obs_fast[name].dtype == "<U40":
            assert np.all(new_obs_fast[name] == observations[break_indx:][name])
        # Otherwise should be number-like
        else:
            assert np.allclose(new_obs_fast[name], observations[break_indx:][name])
# Didn't need to go by column, the observations after restart
# match the ones that were taken all at once.
else:
    assert np.all(new_obs_fast == observations[break_indx:])

AssertionError: 

In [13]:
name

'target_id'

In [14]:
new_obs["target_id"]

ObservationArray([2212, 2213, 2214, ..., 4194, 4195, 4196], shape=(1359,))

In [15]:
observations[break_indx:]["target_id"]


ObservationArray([2212, 2213, 2214, ..., 4194, 4195, 4196], shape=(1359,))